# Logic Field Theory – Stage 9: Asymmetric GHZ State and the Born Rule

This notebook demonstrates how Logic Field Theory (LFT) models the collapse dynamics of an asymmetric GHZ state:
\[
|\psi\rangle = \alpha|000\rangle + \beta|111\rangle, \quad \text{where } |\alpha|^2 \neq |\beta|^2
\]
We test whether LFT's DAG-based collapse simulation, modified to include amplitude-squared weighting, can reproduce the Born rule probabilities across a variety of asymmetries and logical temperatures (β).


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import random
import networkx as nx


In [ ]:
dag = nx.DiGraph()
dag.add_edge("s_aGHZ", "s_000")
dag.add_edge("s_aGHZ", "s_111")

D_values = {
    "s_aGHZ": 1.6,
    "s_000": 0.0,
    "s_111": 0.0
}


In [ ]:
def weighted_collapse(dag, D_values, start_node, weights, beta=1.0, trials=1000):
    outcomes = list(weights.keys())
    counts = {state: 0 for state in outcomes}

    for _ in range(trials):
        numerators = [weights[o] * np.exp(-beta * D_values[o]) for o in outcomes]
        probs = numerators / np.sum(numerators)
        result = random.choices(outcomes, weights=probs)[0]
        counts[result] += 1

    return pd.DataFrame({
        "Outcome": list(counts.keys()),
        "Frequency": list(counts.values()),
        "Empirical Probability": [v / trials for v in counts.values()],
        "Expected Born Probability": [weights[o] for o in outcomes]
    })


In [ ]:
cases = [
    {"alpha2": 0.25, "beta2": 0.75},
    {"alpha2": 0.33, "beta2": 0.67},
    {"alpha2": 0.50, "beta2": 0.50},
    {"alpha2": 0.67, "beta2": 0.33},
    {"alpha2": 0.75, "beta2": 0.25},
]

results = []
for case in cases:
    weights = {"s_000": case["alpha2"], "s_111": case["beta2"]}
    df = weighted_collapse(dag, D_values, "s_aGHZ", weights, beta=1.0, trials=1000)
    df["|alpha|^2"] = case["alpha2"]
    results.append(df)

final_df = pd.concat(results).reset_index(drop=True)
final_df


In [ ]:
import seaborn as sns
sns.set(style="whitegrid")

plt.figure(figsize=(8, 5))
sns.scatterplot(data=final_df, x="Expected Born Probability", y="Empirical Probability", hue="Outcome", s=100)
plt.plot([0, 1], [0, 1], "--", color="gray")
plt.title("LFT vs. Born Rule for Asymmetric GHZ")
plt.xlabel("Expected Born Probability (|α|² or |β|²)")
plt.ylabel("Empirical LFT Probability")
plt.legend()
plt.show()
